# First Reinforcement Learning Algorithms:
- Tic Tac Toe game, with Value Function: This implementation required a high number of exploration before starting exploitation in order to avoid the agents to learn from always the same states

In [1]:
import numpy as np
import random

In [2]:
### Environmnent

def flatten(grid):
    return tuple(grid[i,j] for i in range(3) for j in range(3))

def is_finished(grid):
    for i in range(0,3):
        if list(grid[i,:]) in [[1,1,1],[-1,-1,-1]]:
            return True

        if list(grid[:,i]) in [[1,1,1],[-1,-1,-1]]:
            return True
            
        diag_1 = [grid[0,0],grid[1,1],grid[2,2]]
        diag_2 = [grid[0,2],grid[1,1],grid[2,0]]
        
        if diag_1 in [[1,1,1],[-1,-1,-1]] or diag_2 in [[1,1,1],[-1,-1,-1]]:
            return True
        
        if len(grid.nonzero()[0])==9:
            return True
        return False

class Morpion(object):
    def __init__(self):
        self.field = np.zeros((3,3))
        
    def is_finished(self,state):
        for i in range(0,3):
            if list(self.field[i,:]) in [[1,1,1],[-1,-1,-1]]:
                return True,state,1
            if list(self.field[:,i]) in [[1,1,1],[-1,-1,-1]]:
                return True,state,1
    
        diag_1 = [self.field[0,0],self.field[1,1],self.field[2,2]]
        diag_2 = [self.field[0,2],self.field[1,1],self.field[2,0]]    
        if diag_1 in [[1,1,1],[-1,-1,-1]] or diag_2 in [[1,1,1],[-1,-1,-1]]:
            return True,state,1
        
        if len(self.field.nonzero()[0])==9:
            return True,state,0.5
        
        return False,state,0
    
    def step(self,state,move,player,render = False):
        n_state = state.copy()
        n_state[move] = player.value
        self.field = n_state.copy()
        return game.is_finished(n_state)
        
    def reset(self):
        state = np.zeros((3,3))
        self.field = state.copy()
        return state
        
    def display(self):
        print(self.field)

In [3]:
### Agent ###
class Agent(object):
    def __init__(self, is_human, shape, trainable = True):
        self.is_human = is_human
        self.size = shape
        self.trainable = trainable
        self.eps = 0.99
        self.history = []
        self.known_states = []
        self.V = {}
        self.rewards = []
        self.win_nb = 0
        self.lose_nb = 0
        self.draw_nb = 0
        self.value = 0
        self.show = False
        
    def greedy_step(self,state):
        actions = [(i,j) for i in range(0,3) for j in range(0,3) if state[(i,j)] == 0]
        vmin = None
        vi = None
        
        state = np.array(self.value * state)
        for i in range(len(actions)):
            a = actions[i]
            temp = np.zeros((3,3))
            temp[a] = 1
            fl = flatten( - 1 * ( temp + state ) )
            if  fl not in self.V:
                self.V[fl] = 0
                
            if self.show:
                print(fl)
                print(self.V[fl])
                
            if vmin is None or vmin > self.V[fl]:
                vmin = self.V[fl]
                vi = i
        return actions[vi]
    
    
    def play(self,state):
        if self.is_human:
            inp = input(prompt = 'x y')
            move = (int(inp.split()[0]),int(inp.split()[1]))
        else:
            if np.random.random() < self.eps:
                actions = [(i,j) for i in range(0,3) for j in range(0,3) if state[(i,j)] == 0]
                move = random.choice(actions)
            else:
                move = self.greedy_step(state)
        return move
                
    def add_transition(self,n_tuples):
        self.history.append(n_tuples)
    
    
    def train(self):
        if not self.trainable or self.is_human:
            return
        
        for transition in reversed(self.history):
            s, a, r, sp = transition
            
            if r == 0:
                fl_s = flatten(self.value * s)
                fl_sp = flatten(self.value * sp)
                if fl_s not in self.V:
                    self.V[fl_s] = 0 
                if fl_sp not in self.V:
                    self.V[fl_sp] = 0
                self.V[fl_s] = self.V[fl_s] + 0.01 * ( self.V[fl_sp] - self.V[fl_s] )
            
            else:
                fl_s = flatten(self.value * s)
                if fl_s not in self.V:
                    self.V[fl_s] = 0 
                self.V[fl_s] = self.V[fl_s] + 0.01 * ( r - self.V[fl_s])
        
        self.history = []

In [4]:
### Game ###

def play(game,p1,p2,train = True):
    state = game.reset()
    p1.history = []
    p2.history = []
    players = [p1,p2]
    random.shuffle(players)
    players[0].value = 1
    players[1].value = -1
    
    p=0
    done = False
    
    while done is False:
        
        if players[p%2].is_human:
            game.display()
        
        action = players[p%2].play(state)
        done,n_state,reward = game.step(state,action,players[p%2])
        
        if done:
            players[p%2].win_nb +=1 if reward == 1 else 0
            players[(p + 1)%2].lose_nb +=1 if reward == 1 else 0
            players[p%2].draw_nb +=1 if reward == 0.5 else 0 
            players[(p + 1)%2].draw_nb +=1 if reward == 0.5 else 0
            
        if p != 0:
            s, a, r, _ = players[(p + 1) % 2].history[-1]
            if reward==0.5:
                players[(p + 1) % 2].history[-1] = ( s, a, reward , np.array(n_state))
            else:
                players[(p + 1) % 2].history[-1] = ( s, a, reward * -1 , np.array(n_state))
        
        players[p%2].add_transition((np.array(state), action, reward, None))
        state = n_state.copy()
        
        p += 1
    
    if p1.show:
        print(p1.history)
    if train:
        p1.train()
        p2.train()
        

In [15]:
### Script for training ###
game = Morpion()
p1 = Agent(is_human = False,shape = (3,3),trainable = True)
p2 = Agent(is_human = False,shape = (3,3),trainable = True)

In [18]:
n = 1000000

for i in range(n):
    if i%10000==0:
        p1.eps = max(p1.eps*0.99,0.05)
        p2.eps = max(p2.eps*0.99,0.05)
        print('{}/{}, new epsilon: {}'.format(i,n,p1.eps),end = '\r')
        
    play(game,p1,p2,train = True)

In [19]:
p1.win_nb,p1.lose_nb,p1.draw_nb,p1.eps,p2.win_nb,p2.lose_nb,p2.draw_nb,p1.eps

(842764, 914977, 1242259, 0.05, 914977, 842764, 1242259, 0.05)

In [28]:
p1.show = False
h1 = Agent(is_human = True,shape = (3,3),trainable = True)
play(game,p1,h1)

[[0. 1. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
x y1 1
[[ 0.  1.  0.]
 [ 0. -1.  0.]
 [ 1.  0.  0.]]
x y2 2
[[ 1.  1.  0.]
 [ 0. -1.  0.]
 [ 1.  0. -1.]]
x y0 2


In [30]:
{1:{'epochs':4000000, 'decay':'0.99 every 20000 steps','optimal_behaviour': True}
,2:{'epochs':2000000, 'decay':'0.99 every 10000 steps','optimal_behaviour': True}}

{1: {'epochs': 4000000,
  'decay': '0.99 every 20000 steps',
  'optimal_behaviour': True},
 2: {'epochs': 2000000,
  'decay': '0.99 every 10000 steps',
  'optimal_behaviour': True}}